<a href="https://colab.research.google.com/github/Jinw2/Yanjin-Wang/blob/STA365-homework/sta365_hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
## Question 1
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Colab/hw5 datasets.csv")
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   Year  Market cap ($B)  Revenue ($B)  Earnings ($B)  P/E ratio  P/S ratio  \
0  2022           193.01         23.18           7.82       31.3       8.33   
1  2021           200.31         23.22           9.12       26.5       8.63   
2  2020           159.88         19.20           6.14       33.8       8.32   
3  2019           147.47         21.28           8.01       24.8       6.93   
4  2018           136.21         21.02           7.81       23.2       6.48   

   P/B ratio  Operating Margin (%)  EPS ($)  Shares Outstanding ($B)  \
0      -32.2                 33.76     8.42                     0.73   
1      -43.5                 39.31    10.11                     0.74   
2      -20.4                 31.97     6.35                     0.74   
3      -18.0                 38.04     7.98                     0.74   
4      -21.8                 37.17  

In [4]:
## part 1
import pymc as pm; import numpy as np
X = df['Year'].values.reshape(-1, 1)
y = df['P/B ratio'].values.reshape(-1, 1)

n, p = X.shape[0], 1
with pm.Model() as MLR:
    betas = pm.MvNormal('betas', mu=np.zeros((p,1)), cov=np.eye(p), shape=(p,1))
    sigma = pm.TruncatedNormal('sigma', mu=1, sigma=1, lower=0) # half normal
    y = pm.Normal('y', mu=pm.math.dot(X, betas), sigma=sigma, observed=y)

with MLR:
    idata = pm.sample()

In [5]:
## part 2
import pymc as pm; import numpy as np
X = df['Year'].values.reshape(-1, 1)
y = df['P/B ratio'].values.reshape(-1, 1)

n, p = X.shape[0], 1
with pm.Model() as MLR:
    betas = pm.MvNormal('betas', mu=np.zeros((p,1)), cov=np.eye(p), shape=(p,1))
    sigma = pm.HalfNormal('sigma', sigma=1)
    y = pm.Normal('y', mu=pm.math.dot(X, betas), sigma=sigma, observed=y)

with MLR:
    idata = pm.sample()

In [6]:
## Question 3 part 1
import numpy as np; from scipy import stats
p=df.shape[1]
Psi=np.eye(p)
a_cov = stats.invwishart(df=p+2, scale=Psi).rvs(1)
n=1000
y=stats.multivariate_normal(mean=np.zeros(p), cov=a_cov).rvs(size=n)

In [7]:
with pm.Model() as MNV_LKJ:
    packed_L = pm.LKJCholeskyCov("packed_L", n=p, eta=2.0,
                                 sd_dist=pm.Exponential.dist(1.0, shape=2), compute_corr=False)
    L = pm.expand_packed_triangular(p, packed_L)
    # Sigma = pm.Deterministic('Sigma', L.dot(L.T)) # Don't use a covariance matrix parameterization
    mu = pm.MvNormal('mu', mu=np.array(0), cov=np.eye(p), shape=p);
    # y = pm.MvNormal('y', mu=mu, cov=Sigma, shape=(n,1), observed=y)
    # Figure out how to parameterize this with a Cholesky factor to improve computational efficiency
with MNV_LKJ:
    idata = pm.sample()

In [8]:
with pm.Model() as MNV_LKJ:
    packed_L = pm.LKJCholeskyCov("packed_L", n=p, eta=2.0,
                                 sd_dist=pm.Exponential.dist(1.0, shape=2), compute_corr=False)
    L = pm.expand_packed_triangular(p, packed_L)
    # Sigma = pm.Deterministic('Sigma', L.dot(L.T)) # Don't use a covariance matrix parameterization
    mu = pm.MvNormal('mu', mu=np.array(0), chol=L, shape=p);
    # y = pm.MvNormal('y', mu=mu, cov=Sigma, shape=(n,1), observed=y)
    # Figure out how to parameterize this with a Cholesky factor to improve computational efficiency
with MNV_LKJ:
    idata = pm.sample()

In [9]:

with pm.Model() as MNV_LKJ:
    packed_L = pm.LKJCholeskyCov("packed_L", n=p, eta=2.0,
                                 sd_dist=pm.Exponential.dist(1.0, shape=2), compute_corr=False)
    L = pm.expand_packed_triangular(p, packed_L)
    # Sigma = pm.Deterministic('Sigma', L.dot(L.T)) # Don't use a covariance matrix parameterization
    mu = pm.Normal('mu', mu=0, tau = 1, shape=p);
    y = pm.MvNormal('y', mu=mu, chol=L, shape=(n,1), observed=y)
    # Figure out how to parameterize this with a Cholesky factor to improve computational efficiency
with MNV_LKJ:
    idata = pm.sample()